In [1]:
import pandas as pd

# Load 1.2

In [2]:
destination_path = 'C:/Users/chiny/cjwen/ITI110/'

In [3]:
# df_1_1 = pd.read_csv(destination_path + '1.1 aggregated_hourly_dataset_02112024.csv')
df_1_2 = pd.read_csv(destination_path + '1.2 aggregated_hourly_dataset_without_block_id_02112024.csv')


In [4]:
df_1_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19864 entries, 0 to 19863
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                19864 non-null  object 
 1   month               19864 non-null  int64  
 2   hour                19864 non-null  int64  
 3   count_LCLid         19864 non-null  int64  
 4   count_block_id      19864 non-null  int64  
 5   sum_energy_by_hour  19864 non-null  float64
 6   tstp                19864 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 1.1+ MB


In [38]:
df_1_1['tstp'] = pd.to_datetime(df_1_1['tstp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_1_2['tstp'] = pd.to_datetime(df_1_2['tstp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')



In [39]:
df_1_2.head(2)

,date,month,hour,count_LCLid,count_block_id,sum_energy_by_hour,tstp
0,2011-11-23,11,9,2,2,1.130,2011-11-23 09:00:00
1,2011-11-23,11,10,6,4,1.508,2011-11-23 10:00:00


# Add avg_energy per LCLid column

In [40]:
# df_1_1["avg_energy_per_lclid"]=df_1_1["sum_energy_by_hour"]/df_1_1["count_LCLid"]

In [41]:
df_1_2["avg_energy_per_lclid"]=df_1_2["sum_energy_by_hour"]/df_1_2["count_LCLid"]

# merge holidays

In [47]:
holidays = pd.read_csv(destination_path + "Raw_data/uk_bank_holidays.csv")

In [48]:
holidays["holiday"]=1

In [49]:
holidays

,Bank holidays,Type,holiday
0,2012-12-26,Boxing Day,1
1,2012-12-25,Christmas Day,1
2,2012-08-27,Summer bank holiday,1
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday),1
4,2012-04-06,Spring bank holiday (substitute day),1
5,2012-07-05,Early May bank holiday,1
6,2012-09-04,Easter Monday,1
7,2012-06-04,Good Friday,1
8,2012-02-01,New Year?s Day (substitute day),1
9,2013-12-26,Boxing Day,1


In [50]:
holidays["Bank holidays"] = pd.to_datetime(holidays["Bank holidays"], format='%Y-%m-%d', errors='coerce')

In [51]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Bank holidays  25 non-null     datetime64[ns]
 1   Type           25 non-null     object        
 2   holiday        25 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 732.0+ bytes


In [52]:
# df_1_1_merge_hols = pd.merge(df_1_1, holidays, left_on = "tstp", right_on = "Bank holidays", how = "left")
df_1_2_merge_hols = pd.merge(df_1_2, holidays, left_on = "tstp", right_on = "Bank holidays", how = "left")



In [54]:
# df_1_1_merge_hols["holiday"].fillna(0,inplace=True)
df_1_2_merge_hols["holiday"].fillna(0,inplace=True)


In [55]:
#verify by checking one of the holidays


,date,month,count_LCLid,count_block_id,sum_energy_by_day,avg_energy_per_lclid,Bank holidays,Type,holiday
763,2013-12-25,12,5082,112,63990.164991,12.591532,2013-12-25,Christmas Day,1.0


In [56]:
columns_to_drop = ["Bank holidays", "Type"]
# df_1_1_merge_hols.drop(columns=columns_to_drop, axis=1, inplace=True)
df_1_2_merge_hols.drop(columns=columns_to_drop, axis=1, inplace=True)


# Merge weather data

In [57]:
weather = pd.read_csv(destination_path + 'Raw_data/weather_hourly_darksky.csv')

In [58]:
weather['time'] = pd.to_datetime(weather['time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [59]:
df_1_1_merged_df = pd.merge(df_1_1_merge_hols, weather, left_on = "tstp", right_on = "time", how = "left")
df_1_2_merged_df = pd.merge(df_1_2_merge_hols, weather, left_on = "tstp", right_on = "time", how = "left")


In [65]:
df_1_2_merged_df.isnull().sum()

date                     0
month                    0
hour                     0
count_LCLid              0
count_block_id           0
sum_energy_by_hour       0
tstp                     0
avg_energy_per_lclid     0
holiday                  0
visibility               2
windBearing              2
temperature              2
time                     2
dewPoint                 2
pressure                13
apparentTemperature      2
windSpeed                2
precipType               2
icon                     2
humidity                 2
summary                  2
dtype: int64

In [63]:
df_1_2_merged_df.to_csv(destination_path + '2.2 merged_aggregated_hourly_without_block_id.csv', index=False)